In [1]:
#import libraries
import sys
import os
import time
import datetime as dt
import numpy as np
import pandas as pd
import norgatedata as ng
from functions import trend_filter, position_size_atr, signal_filter, select_positions_1, \
                      exposure_summary, process_universe_factors, filter_ranks, step_activation, \
                      construct_trade_df, read_prior_activations
from ib_connect import main as positions

sys.path.append(os.path.abspath('../fin_data'))
from trade_logic.indicators import stock_beta, stock_volatility, atr, ewma
from utils.date_functions import last_business_day, time_elapsed
from utils.postgresql_data_query import get_bars, get_bm_bars, get_effective_dates
from data_updater.norgate_data import NorgateData

start_time = time.time()

[2024-10-06 13:24:02.233880] INFO: Norgate Data: NorgateData package v1.0.74: Init complete
connected to: dbmaster


In [2]:
## set up parameters
bm_col = '$SPX'
lo_rank = 50
hi_rank = 50
w_offset = 0
d_offset = max(1, w_offset * 7)
beta_n = 36
vola_n = 20
atr_n = 20
ewma_stop = 128
ewma_n = 128
dgt = 2
cols = ['Date','Open','High','Low','Close']
factor_cols = ['ticker','factor_date','factor_definition_id','rank']
model_factors = [85, 86, 107, 7]
activation_factor = 85
activation_threshold = 95
activation_mult = 2
risk_factor = 0.001
sector_max = 0.20
max_positions = 200

excluded_tickers = ['VZIO']

client_id = 1
account_number = "DU3208934"
path = '/Users/VadimKovshov/Dropbox/INVESTMENTS/EVALUTE/STOCKS/MODEL_OUTPUTS/MOMENTUM_MODEL/'

use_postgresql = True
end = last_business_day(offset=d_offset)
start = last_business_day(offset=d_offset + 400)
# current_date = dt.date.today()
# prior_model_date = '2024-09-16'
print(f'Historical range: {end.strftime("%Y-%m-%d")} to {start.strftime("%Y-%m-%d")}')

Historical range: 2024-10-04 to 2023-09-01


In [3]:
# get most recent positions from IB
positions, account_value = positions(client_id, account_number, path)

if positions.empty:
    print('No current positions found')
else:
    print(f'Current positions: {len(positions)}')
    positions.head(3)

if account_value:
    print(f'Account value: {account_value}')
else:
    account_value = 602600
    print(f'Using nominal account value: {account_value}')

position_tickers = positions['ticker'].tolist()
positions.head(3)

Current positions: 21
Account value: 603538.23


,account,ticker,position,average_cost
0,DU3208934,TRGP,150.0,146.378327
1,DU3208934,ENVA,450.0,78.550274
2,DU3208934,IRM,210.0,114.507636


In [4]:
# get universe factors for a given week
eff_date, pr_date = get_effective_dates(offset_0=w_offset)
model_date = eff_date.strftime("%Y-%m-%d")
prior_model_date = pr_date.strftime("%Y-%m-%d")

print(f'Factor dates now: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')

ranks_now = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                     hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols)

positions_now = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                     hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols, tickers=position_tickers)

ranks_now = pd.concat([ranks_now, positions_now])
ranks_now.drop_duplicates(subset=['ticker'], inplace=True)

# get universe factors for one week prior
eff_date, pr_date = get_effective_dates(offset_0=w_offset+1)
print(f'Factor dates prior: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')

ranks_prior = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                       hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols)

positions_prior = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                       hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols, tickers=position_tickers)

ranks_prior = pd.concat([ranks_prior, positions_prior])
ranks_prior.drop_duplicates(subset=['ticker'], inplace=True)

Factor dates now: 2024-10-04, 2024-09-27
Test universe: 1772
Test universe: 21
Factor dates prior: 2024-09-27, 2024-09-20
Test universe: 1787
Test universe: 21


In [5]:
# Obtain prior model output
prior_output = f'{path}momo_model_output_{prior_model_date}.csv'
prior_activations_df = read_prior_activations(prior_output)
# Filter the DataFrames
filter_used = 1
df_now = filter_ranks(ranks_now, filter=filter_used)
df_prior = filter_ranks(ranks_prior, filter=filter_used)

# Merge with prior activations to include the prior activation values
df_prior = pd.merge(df_prior, prior_activations_df, on='ticker', how='left')

combined_df = pd.merge(df_now, df_prior, on='ticker', how='left', suffixes=('', '_prior'), indicator=True)
# Assign labels based on the merge result
combined_df['label'] = combined_df['_merge'].map({'left_only': 'new', 'both': 'old'})

# Check if ticker is in position_tickers, and assign 'position' label
combined_df['label'] = np.where(combined_df['ticker'].isin(position_tickers), 'position', combined_df['label'])

# Handle missing activation_value or prior signal
combined_df['activation_value'] = combined_df['activation_value'].fillna(1)  # Set a default activation value if missing
combined_df[f'f_{activation_factor}_prior'] = combined_df[f'f_{activation_factor}_prior'].fillna(0)  # Default for prior signals

# Calculate the activation signal  
combined_df['activation'] = combined_df.apply(
    lambda row: step_activation(
        signal_now=row[f'f_{activation_factor}'], 
        signal_prior=row[f'f_{activation_factor}_prior'] if pd.notnull(row[f'f_{activation_factor}_prior']) else 0,
        activation_value=row['activation_value'],
        label=row['label'],
        extreme_threshold=activation_threshold, 
        activation_mult=activation_mult
    ), axis=1
)

combined_df = combined_df.drop(columns=['_merge'])
columns_to_keep = list(df_now.columns) + ['label', 'activation']
combined_df = combined_df[columns_to_keep]

# Optionally, reset the index
combined_df = combined_df.reset_index(drop=True)

# Filter tickers and limit to max_positions
tickers = (
    combined_df
    .loc[~combined_df['ticker'].isin(excluded_tickers), 'ticker']
    .unique()
    .tolist()[:max_positions]
)

print(f'Filtered tickers: {len(tickers)}')

Filtered tickers: 83


In [6]:
# get industry data
ng_obj = NorgateData(symbols= tickers, database='US Stocks')
ng_obj.process_metadata()
industry_df = ng_obj.df_result['metadata'].rename(columns={'symbol':'ticker', 'security_name':'name', 'GICS_level_1': 'sector', 'GICS_level_2': 'industry_group',
                                                            'GICS_level_3': 'industry', 'GICS_level_4': 'sub_industry'}).reset_index(drop=True)
industry_df = industry_df[['ticker', 'sector', 'industry_group', 'industry', 'sub_industry', 'name']]

sorted_df = pd.merge(combined_df, industry_df, on='ticker', how='left')

print (f'Sector tickers: {len(tickers)}')

Sector tickers: 83


In [7]:
# get benchmark data
if use_postgresql:
    bm_df = get_bm_bars(bm_col,start,end) # uses saved PostgreSQL data
else:
    bm_df = ng_obj.daily_bars('$SPX', start, end, interval='D') # uses NorgateData data
    bm_df = bm_df[cols]
    bm_df['Ticker'] = bm_col

print(f'Benchmark data: {len(bm_df)}')

Benchmark data: 275


In [8]:
exposure = trend_filter(bm_df, _c='Close')
exposure = 1 #exposure[-1]
print(f'Exposure: {exposure}')

Exposure: 1


In [9]:
# get stock data and process signals
df_f = pd.DataFrame()
i = 0
for t in tickers:
    i += 1
    try:
        if use_postgresql:
            st_df = get_bars([t],start,end,backtest=False) # uses saved PostgreSQL data
        else:
            st_df = ng_obj.daily_bars(t, start, end, interval='D') # uses NorgateData data
            st_df = st_df[cols]
            st_df['Ticker'] = t

        beta_value = round(stock_beta(df=st_df, _c='Close', bm_df=bm_df, bm='Close', n=beta_n),dgt)
        vola_value = stock_volatility(df=st_df, _c='Close', n=vola_n, annualized=False)
        atr_value = round(atr(df=st_df, _h='High', _l='Low', _c='Close', n=atr_n).iloc[-1],2)
        activation_value = combined_df.loc[combined_df['ticker'] == t, 'activation'].values[0]
        ewma_value = round(ewma(df=st_df, _c='Close', _n=(ewma_stop / activation_value), _mp=(ewma_stop / activation_value)).iloc[-1],dgt)
        last_close = st_df['Close'].iloc[-1]
        signal_value = signal_filter(df=st_df, _h='High', _l='Low', _c='Close', n=ewma_n, pct_change_threshold=0.15, period=90)
        atr_position_size = activation_value * signal_value * position_size_atr(account_value, risk_factor, exposure, atr_value) // 10 * 10
        dollar_position = atr_position_size * st_df['Close'].iloc[-1]
        pct_position = round(dollar_position / account_value,dgt)

        data = {
            'date': st_df['Date'].iloc[-1],
            'ticker': t,
            'beta': beta_value,
            'volatility': vola_value,
            'atr': atr_value,
            'last_close': last_close,
            'ewma_stop': ewma_value,
            'signal': signal_value,
            'new_position': atr_position_size,
            'dollar_position': dollar_position,
            'pct_position': pct_position,
            'activation_value': activation_value
        }
        df = pd.DataFrame([data])
        df_f = pd.concat([df_f,df],ignore_index=True)
        if i % 20 == 0:
            print(f'Completed: {i}, {t}, {time_elapsed(start_time)}')
    except Exception as e:
        print(f"Error processing {t}: {e}")  # Print out the exception for debugging
        continue

print(f'Completed: {i}, {t}, {time_elapsed(start_time)}')

Completed: 20, IRM, 1 minutes and 21 seconds
Completed: 40, JLL, 1 minutes and 30 seconds
Completed: 60, IIPR, 1 minutes and 38 seconds
Completed: 80, BRO, 1 minutes and 47 seconds
Completed: 83, DTM, 1 minutes and 48 seconds


In [11]:
# final filter
df_f_final = pd.merge(sorted_df, df_f, on='ticker', how='left')
df_f_final = df_f_final[df_f_final['signal'] == 1].reset_index(drop=True).copy()
df_f_final = df_f_final.drop(columns=['factor_date']).reset_index(drop=True)

In [12]:
# create model output and trade dataframes
selected_positions_df = select_positions_1(df_f_final, max_exposure=exposure, max_sector_allocation=sector_max).sort_values(by='ticker')
trade_df = construct_trade_df(positions, selected_positions_df).sort_values(by='ticker')

In [13]:
summary_positions = selected_positions_df[selected_positions_df['new_position'] != 0]
styled_summary = exposure_summary(summary_positions)

print(f'Completed: {time_elapsed(start_time)}')
styled_summary

Total exposure: 0.97
Completed: 2 minutes and 10 seconds


,sector,sector_exp,industry,industry_exp,ticker,exposure,name
0,Financials,0.19,Capital Markets,0.13,JEF,0.09,Jefferies Financial Group Inc Common
1,Financials,0.19,Capital Markets,0.13,PIPR,0.04,Piper Sandler Companies Common
2,Financials,0.19,Consumer Finance,0.06,ENVA,0.06,Enova International Inc Common
3,Real Estate,0.18,Real Estate Management & Development,0.11,NMRK,0.07,Newmark Group Inc Class A Common
4,Real Estate,0.18,Real Estate Management & Development,0.11,JLL,0.04,Jones Lang LaSalle Inc Common
5,Real Estate,0.18,Specialized REITs,0.04,IRM,0.04,Iron Mountain Inc Common
6,Real Estate,0.18,Office REITs,0.03,SLG,0.03,SL Green Realty Common
7,Information Technology,0.13,Software,0.07,ACIW,0.04,ACI Worldwide Inc Common
8,Information Technology,0.13,Software,0.07,FICO,0.03,Fair Isaac Corp Common
9,Information Technology,0.13,"Electronic Equipment, Instruments & Components",0.06,GLW,0.06,Corning Inc Common


In [14]:
# save output to csv
selected_positions_df.to_csv(f'{path}momo_model_output_{model_date}.csv', index=False)
trade_df.to_csv(f'{path}momo_trade_file_{model_date}.csv', index=False)
trade_df

,ticker,label,position,shares_to_trade,new_position,average_cost,dollar_position,pct_position,ewma_stop,name,sector
7,ACIW,position,480.0,20.0,500.0,48.911510,25945.0,0.04,42.38,ACI Worldwide Inc Common,Information Technology
10,ADUS,position,220.0,0.0,220.0,133.387966,27788.2,0.05,119.44,Addus Homecare Corp Common,Health Care
22,AEM,NaN,290.0,-290.0,0.0,81.412946,NaN,NaN,NaN,NaN,NaN
21,AGI,position,1050.0,20.0,1070.0,19.805052,20961.3,0.03,17.46,Alamos Gold Inc Class A Common,Materials
23,ALKT,NaN,580.0,-580.0,0.0,31.295052,NaN,NaN,NaN,NaN,NaN
24,CACI,NaN,70.0,-70.0,0.0,488.568117,NaN,NaN,NaN,NaN,NaN
15,CALM,new,0.0,290.0,290.0,NaN,24525.3,0.04,66.47,Cal-Maine Foods Inc Common,Consumer Staples
20,CNK,new,0.0,850.0,850.0,NaN,22746.0,0.04,23.27,Cinemark Holdings Inc Common,Communication Services
16,COKE,position,10.0,10.0,20.0,1304.090050,25841.8,0.04,1134.49,Coca-Cola Consolidated Inc Common,Consumer Staples
17,CSWI,position,40.0,10.0,50.0,315.322552,18742.0,0.03,294.82,CSW Industrials Inc Common,Industrials
